<a href="https://colab.research.google.com/github/AIAerospace/DeepLearning/blob/main/8-CNN/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

### 1. Utilizar un modelo pre-entrenado:

In [ ]:
import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16


image_orig = load_img('tren.jpg', target_size=(224, 224))
image_orig = img_to_array(image_orig)
image_orig = image.reshape((1, image_orig.shape[0], image_orig.shape[1], image_orig.shape[2]))

# preparamos la imagen para el modelo y cargamos el modelo
image = preprocess_input(image_orig)
model = VGG16(weights='imagenet', include_top=True)
# predicción
yhat = model.predict(image)           # Probabilidades
label = decode_predictions(yhat)      # Etiquetas
label = label[0][0]

print('%s (%.2f%%)' % (label[1], label[2]*100))

### 2. Feature transfer:

Vamos a eliminar el clafificador al final de la red original, sustituirlo por el nuestro y entrenar.

In [ ]:
from keras.datasets import cifar10
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Flatten
from keras.utils import to_categorical
import matplotlib.pyplot as plt

# CIFAR10 Dataset
(train_data, train_labels), (val_data, val_labels) = cifar10.load_data()

# Normalizar pixel values to [0, 1]
train_data = train_data / 255.0
val_data = val_data / 255.0

# One-hot encoding
train_labels = to_categorical(train_labels, num_classes=10)
val_labels = to_categorical(val_labels, num_classes=10)

In [ ]:
# Cargar el modelo VGG original, si el clasificador
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = base_model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
pretrain_model = Model(inputs=base_model.input, outputs=predictions)

# Congelar las capas del modelo pre-entrenado
for layer in base_model.layers:
    layer.trainable = False

# Compilar
pretrain_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
pretrain_model.summary()


In [ ]:
# Fit
history_pretrain = pretrain_model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))


### 3. Fine tunning:

In [ ]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
pretrain_model = Model(inputs=base_model.input, outputs=predictions)
pretrain_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Congelar las capas
for layer in base_model.layers:
    layer.trainable = False

# Descongelar las cuatro ultimas
for layer in pretrain_model.layers[-4:]:
    layer.trainable = True

# Compilar
pretrain_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar
history_finetune = pretrain_model.fit(train_data, train_labels, epochs=15, validation_data=(val_data, val_labels))

